In [1]:
""" Alpaca Paper Trade with the 1st LSTM """

import os

if "src" not in os.listdir():
    os.chdir("../../../")

import json

import time
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone
import pickle

import pandas_market_calendars as mcal
from keras.models import load_model

from src.lib.alpaca_historical import AlpacaData
from src.lib.alpaca_paper import AlpacaTrader
from src.lib.activations import negative_softmax
from src.lib.price_history import transform_price_history, PriceHistory
from src.lib.stock_dataset import StockDataset

In [2]:
path = "results/1_lstm_msla/"
keys = "alpaca_config.json"
isPaper = True

In [3]:
def now_utc():
    return datetime.now().astimezone(tz=timezone("UTC"))


def get_next_trading_minute():
    nyse = mcal.get_calendar("NYSE")
    now = now_utc()
    lookahead = timedelta(days=0)
    while True:
        lookahead += timedelta(hours=1)
        schedule = nyse.schedule(start_date=now, end_date=now + lookahead)
        valid_minutes = mcal.date_range(schedule, frequency="1T")
        where_future = np.where(valid_minutes > now)[0]
        if len(where_future) > 0:
            break

    # return valid_minutes[where_future[0]]
    return (now + timedelta(minutes=1)).replace(second=0)

In [4]:
api_data = AlpacaData(keys)
api_trade = AlpacaTrader(keys, isPaper)
model = load_model(
    os.path.join(path, "model.h5"),
    custom_objects={
        "negative_softmax": negative_softmax
    }
)

with open(os.path.join(path, "config.json"), "r") as f:
    config = json.load(f)

n_time_steps = config["n_time_steps"]
symbols = config["symbols"]
target_column = config["target_column"]

with open(os.path.join(path, "scaler.pkl"), "rb") as f:
    scaler = pickle.load(f)

In [6]:
def trade_loop():
    # Sleep until next trading time
    next_trade_time = get_next_trading_minute()
    sleep_seconds = (next_trade_time - now_utc()).total_seconds() + 1
    print("Sleeping for %.2f" % sleep_seconds)
    time.sleep(sleep_seconds)

    # Get the last 64 candles
    data_start_time = next_trade_time - timedelta(minutes=n_time_steps + 1 + 60) 
    price_histories = []
    for symbol in symbols:
        history = api_data.get_bars(symbol, data_start_time, next_trade_time, log=False)
        if type(history) is str:
            print(history)
        price_history = PriceHistory(history, symbol)
        # TODO accept **kwargs here
        price_history = transform_price_history(price_history)
        price_histories.append(price_history)
    
    # Process the candes
    dataset = StockDataset(price_histories, target_column, n_time_steps)
    dataset.apply_standard_scaler(scaler)
    X = dataset.prediction_X(n_time_steps)
    
    # Model makes predictions
    preds = model.predict(X)[0]
    next_holdings = dict(zip(symbols, preds))
    print("Desired holdings", next_holdings)
    print(api_trade.update_holdings(next_holdings, verbose=True))

while True:
    trade_loop()

Sleeping for 3.88
Desired holdings {'SPY': 0.54154485, 'SPXS': -0.45415536}
sell SPY 9467.699468457206
sell SPXS 45334.25728422984
[<Response [200]>, <Response [422]>]
Sleeping for 57.89


KeyboardInterrupt: 